# Import Utili

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from sqlalchemy import create_engine
import pymysql

# Definisco funzioni per caricare e pulire i dati
## Ho 5 funzioni :
  - **readCsv** : leggo il csv 
  - **getCategory** : leggo solo la colonna category normalizzata visto che i record in quella colonna sono in formato json 
  - **getCreator** : leggo solo la colonna creator normalizzata visto che i record in quella colonna sono in formato json 
  - **besteffort_loads** : funzione che mi permette di caricare i json not well formed impostando valori non leggibili a null 
  - **merge** : Prendo in ingresso una DataFrame corrispondente a una colonna con i dati in formato json e la aggiungo al dataset iniziale 

In [3]:
def readCsv(path):
    return pd.read_csv(path)


In [4]:
def getCategory(dataset):
    normalized_category=pd.json_normalize(dataset.category.apply(json.loads))
    normalized_category.columns=normalized_category.columns.map(lambda x: str(x) + '_category')
    return normalized_category

In [5]:
def besteffort_loads(obj):
    try:
        return json.loads(obj)
    except (TypeError, ValueError):
        return {}

In [6]:
def getCreator(dataset):
    normalized_creator=pd.json_normalize(dataset.creator.apply(besteffort_loads))
    normalized_creator.columns=normalized_creator.columns.map(lambda x: str(x) + '_creator')
    return normalized_creator

In [7]:
def merge(dataset,category,name):
    merged_data = pd.merge(dataset,category, left_index=True, right_index=True)
    del merged_data[name]
    return merged_data

# Pulisco il Dataset
 Utilizzando le funzioni prima definite carico e pulisco i dati di un Dataset

In [8]:
dataset=readCsv('csv\Kickstarter000.csv')
category= getCategory(dataset)
dataset= merge(dataset,category,'category')
creator= getCreator(dataset)
dataset= merge(dataset,creator,'creator')
dataset

,backers_count,blurb,converted_pledged_amount,country,country_displayable_name,created_at,currency,currency_symbol,currency_trailing_code,current_currency,...,name_creator,is_registered_creator,chosen_currency_creator,is_superbacker_creator,avatar.thumb_creator,avatar.small_creator,avatar.medium_creator,urls.web.user_creator,urls.api.user_creator,slug_creator
0,76,Dave-Man. A game about getting Dave his coffee.,697,US,the United States,1541608918,USD,$,True,USD,...,Lou Bagel,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/022/482/563/b...,https://ksr-ugc.imgix.net/assets/022/482/563/b...,https://ksr-ugc.imgix.net/assets/022/482/563/b...,https://www.kickstarter.com/profile/294143625,https://api.kickstarter.com/v1/users/294143625...,NaN
1,6,"My Vision ""Shanghai Dreaming Podcast""this seri...",440,US,the United States,1377505616,USD,$,True,USD,...,Sabrina Montgomery,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/007/803/708/7...,https://ksr-ugc.imgix.net/assets/007/803/708/7...,https://ksr-ugc.imgix.net/assets/007/803/708/7...,https://www.kickstarter.com/profile/sabrinamon...,https://api.kickstarter.com/v1/users/109791602...,sabrinamontgomery
2,2,I have always loved cookies and have a great r...,40,US,the United States,1424288631,USD,$,True,USD,...,daniel koert,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/009/587/181/5...,https://ksr-ugc.imgix.net/assets/009/587/181/5...,https://ksr-ugc.imgix.net/assets/009/587/181/5...,https://www.kickstarter.com/profile/1496002217,https://api.kickstarter.com/v1/users/149600221...,NaN
3,73,Kickstarter Campaign for the The Art of Mitch ...,11192,US,the United States,1463615618,USD,$,True,USD,...,Mitch Foust,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/007/143/777/7...,https://ksr-ugc.imgix.net/assets/007/143/777/7...,https://ksr-ugc.imgix.net/assets/007/143/777/7...,https://www.kickstarter.com/profile/999214729,https://api.kickstarter.com/v1/users/999214729...,NaN
4,0,"I want to make this an app that, 33 states and...",0,US,the United States,1508774685,USD,$,True,USD,...,Sylvester S. Harris III,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/018/879/858/a...,https://ksr-ugc.imgix.net/assets/018/879/858/a...,https://ksr-ugc.imgix.net/assets/018/879/858/a...,https://www.kickstarter.com/profile/1461334453,https://api.kickstarter.com/v1/users/146133445...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,33,A Character Driven Illustrated Tarot Deck with...,1913,US,the United States,1449113890,USD,$,True,USD,...,Michele Hartsoe,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/010/617/713/a...,https://ksr-ugc.imgix.net/assets/010/617/713/a...,https://ksr-ugc.imgix.net/assets/010/617/713/a...,https://www.kickstarter.com/profile/796990185,https://api.kickstarter.com/v1/users/796990185...,NaN
3702,194,Cerin is a supernatural black cat that hunts e...,9463,US,the United States,1476826701,USD,$,True,USD,...,Stefanie Black,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/009/364/011/f...,https://ksr-ugc.imgix.net/assets/009/364/011/f...,https://ksr-ugc.imgix.net/assets/009/364/011/f...,https://www.kickstarter.com/profile/creepykawaii,https://api.kickstarter.com/v1/users/191055078...,creepykawaii
3703,5,"Dry clean, Free storage, Free Delivery. Save s...",122,US,the United States,1469493360,USD,$,True,USD,...,Pocket Closet Dry Cleaner,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/013/176/841/2...,https://ksr-ugc.imgix.net/assets/013/176/841/2...,https://ksr-ugc.imgix.net/assets/013/176/841/2...,https://www.kickstarter.com/profile/1497537725,https://api.kickstarter.com/v1/users/149753772...,NaN
3704,1,I want to start a Film that nobody has ever do...,5,US,the United States,1423675038,USD,$,True,USD,...,Camo,NaN,NaN,NaN,https://ksr-ugc.imgix.net/assets/009/559/816/d...,https://ksr-ugc.imgix.net/assets/009/559/816/d...,https://ksr-ugc.imgix.net/assets/009/559/816/d...,https://www.kickstarter.com/profile/1619849742,https://api.kickstarter.com/v1/users/161984974...,NaN


# Connessione al DB

In [ ]:
sqlEngine = create_engine('mysql+pymysql://root:@127.0.0.1/test', pool_recycle=3600)
dbConnection = sqlEngine.connect()

In [ ]:
index='country'
n= dataset['country'].nunique()
country= dataset['country']
plt.hist(country,n)